Plot marker synteny plots using `jcvi.graphics.synteny`. 

Change introduced at line 173 of the submodule: `color = g.score#forward if strand == "+" else backward`. 
This was done to infer block color directly from column 5 of combined BED.

Files generated for each of gambiae chromosomes:
- display config. 
- blocks. Each row is a list of syntenic markers between markers in different species. `.`
- merged bed of amplicon positions in three genomes (note that jcvi does not check for interchromosomal rearrangements)
- extra features - start and end of gambiae chromosomes

Usage of `jcvi.graphics.karyotype` for plotting all chromosomes at once also considered. It requires more convoluted blocks format, pairwise anchors, separate beds and other version of display config. It  does not represent amplicons as blocks, so any amplicons without homology will be invisible. Also, no interchromosomal rearrangements seem to exist in our dataset.

In [1]:
import os
import pandas as pd

In [2]:
# in 
IN_META = '../../data/panel_extended_info.csv'
IN_BED_DIR = '../../../data/phylo_ampl/vobs_genomes/locationsByGenome/'
IN_BED = {
    'Agam': os.path.join(IN_BED_DIR, 'anopheles-gambiae-pestchromosomesagamp4.bed'),
    'Afun': os.path.join(IN_BED_DIR, 'anopheles-funestus-fumozchromosomesafunf3.bed'),
    'Aalb': os.path.join(IN_BED_DIR, 'anopheles-albimanus-steclachromosomesaalbs2.bed')
}
# intermediates
# OUT_BED = 'data/{}.bed'
# ANCHORS = 'data/Agam.{}.anchors.txt'
# SEQIDS = 'data/seqids.txt'
DISP_CONF = 'data/disp.{}.txt'
BLOCKS = 'data/blocks.{}.txt'
COMB_BED = 'data/comb.{}.bed'
CHROM_ENDS = 'data/chrom_ends.{}.bed'
# params
SPP = ['Agam','Aalb','Afun']
SP_CAPTIONS = {
    'Agam': 'gambiae',
    'Afun': 'funestus',
    'Aalb': 'albimanus'
}
ANNOT_COLORS = {
    'exon':'orangered',
    'intron':'gold',
    'intergenic':'royalblue'
}
# agam, afun, aalb
CHROM_ENDS = {
    "2L":(("Agam_2L",1,49364325),("Afun_3",1,45633784),("Aalb_3L",1,33601328)), 
    "2R":(("Agam_2R",1,61545105),("Afun_2",1,56047416),("Aalb_2R",1,51801758)), 
    "3L":(("Agam_3L",1,41963435),("Afun_3",45633828,93833897),("Aalb_3R",1,34682003)), 
    "3R":(("Agam_3R",1,53200684),("Afun_2",56047552,99479338),("Aalb_2L",1,37976048)), 
    "X":(("Agam_X",1,24393108),("Afun_X",1,17661987))
}
AGAM_CHROM_SIZES = {k:v[0][2] for (k,v) in CHROM_ENDS.items()}
# funestus chromosome arms merged in assembly
GAM_TO_FUN_CHROM = {
    "2L":"3R", 
    "2R":"2R", 
    "3L":"3L", 
    "3R":"2L", 
    "X":"X"
}
# size of drawn region for each amplicon
REGSIZE = 1000000

In [3]:
!mkdir -p data

## Metadata

In [4]:
# read metadata
meta = pd.read_csv(IN_META)
# need to recode amplicon IDs from SeekDeep to current
id_recode = meta \
        .set_index('SeekDeep_ID')['Primary_ID'] \
        .to_dict()
id_recode['V0']

'0'

In [6]:
meta['annotation'] = (meta.intergenic == 'Yes').replace({True:'intergenic',False:'exon'})
meta.loc[meta.intron == 'Yes', 'annotation'] = 'intron'
meta.head()

,Primary_ID,ID,SeekDeep_ID,SNP,F,R,chr,start,end,F_deg,...,BDGP6_s_span,CpipJ2_identity,CpipJ2_q_span,CpipJ2_s_span,Comments,Outgroup ampl,conc_multiplier,annotation,start_insert,end_insert
0,0,31_var,V0,high freq,TGTSTACGGTCTGAAGAACATc,TTATCCGGCTCCAAGTTAAGG,2L,10356417,10356608,TGT[G/C]TACGGTCTGAAGAACATc,...,2R:22800075-22800136,0.968253968,1-63,supercont3.30:1160215-1160153,F primer perfect match in Cpip&Aaeg,No,10,intron,10356439,10356587
1,1,33_other,O0,low freq,GAGCGtGCGGCcAAGATG,ACAgACCGACGTTAATGGC,2L,10357458,10357648,GAGCGtGCGGCcAAGATG,...,NaN,0.926315789,1-190,supercont3.30:1159280-1159091,"Cpip - perfect, Aaeg - 5` subst",Yes,10,exon,10357476,10357629
2,2,44_other,O1,low freq,CAGTCAAATTTCCAGACAATCT,CGGAAGTGCATTTGAAGGAAaA,2L,12084495,12084690,CAGTCAAATTTCCAGACAATCT,...,NaN,NaN,NaN,NaN,NaN,NaN,5.8,intergenic,12084517,12084668
3,3,47_other,O2,low freq,GaTATAAATTGTCGATCACACAAACT,TGCATTTATCGTAGTACAATCTCA,2L,13862864,13863055,GaTATAAATTGTCGATCACACAAACT,...,NaN,NaN,NaN,NaN,NaN,NaN,2.6,intergenic,13862890,13863031
4,4,58_cons,C0,high freq,ATGcTBGTCATgATGATGATCT,CCGATCCACGATAAGGAGTAC,2L,18693481,18693714,ATGcT[C/T/G]GTCATgATGATGATCT,...,NaN,0.834042553,1-233,supercont3.137:804142-804374,5`subst in Aaeg&Cpip,Yes,10,exon,18693503,18693693


In [7]:
annot = meta.set_index('Primary_ID')['annotation'].to_dict()
annot['0']

'intron'

In [24]:
# write annotations to original metadata
meta.to_csv(IN_META, index=False)
! head -2 {IN_META}

Primary_ID,ID,SeekDeep_ID,SNP,F,R,chr,start,end,F_deg,R_deg,identical_seq,idenified_lineages,aligned_spp,unaligned_spp,aligned_ampl_length,agam_ampl_length,aligned_insert_length,agam_insert_length,total_snvs,total_indels,insert_snvs,insert_indels,prop_id_lineages,exon,gene,intergenic,intron,mRNA,repeat,utr,AaegL5_identity,AaegL5_q_span,AaegL5_s_span,AgamP3_identity,AgamP3_q_span,AgamP3_s_span,BDGP6_identity,BDGP6_q_span,BDGP6_s_span,CpipJ2_identity,CpipJ2_q_span,CpipJ2_s_span,Comments,Outgroup ampl,conc_multiplier,annotation,start_insert,end_insert
0,31_var,V0,high freq,TGTSTACGGTCTGAAGAACATc,TTATCCGGCTCCAAGTTAAGG,2L,10356417,10356608,TGT[G/C]TACGGTCTGAAGAACATc,TT[A/G]TCCGGCTCCAAGTTAAGG,"[{'AsteI2', 'AsteS1'}]",20.0,21.0,,259,191.0,216.0,148.0,33.0,139.0,30.0,139.0,0.9523809520000001,E016098A;E016099A,AGAP005134,No,Yes,AGAP005134-RA,None,No,0.96969697,1-66,2:204342881-204342946,1.0,1-191,2L:10356418-10356608,0.951612903,1-62,2R:22800075-22800136,0.968253968,1-63,supercont3.30:1160215-1

## Prepare data

In [9]:
# process input BEDs
comb_data = dict()
for sp, bedfile in IN_BED.items():
    sp_data = pd.read_csv(bedfile, 
                          sep='\t', 
                          header=None,
                          names=['chrom','start','end','id','len','strand'])
    # get amplicon name
    sd_ids = sp_data['id'].str.split('=').str.get(-1).str.slice(0,-1)
    # get extraction count
    extr_count = sp_data['id'].str.split('=').str.get(1).str.split(';').str.get(0)
    # replace and prepend with species name
    sp_data['id'] = sp + '_' + sd_ids.replace(id_recode)
    # remove repetitive positions (if any)
    sp_data['extraction'] = extr_count
    sp_data = sp_data[sp_data['extraction'] == '0'].drop(columns=['extraction'])
    comb_data[sp] = sp_data
comb_data['Afun'].head()

,chrom,start,end,id,len,strand
0,2,2538226,2538411,Afun_12,185,+
1,2,10269306,10269539,Afun_16,233,+
2,2,18159344,18159529,Afun_25,185,-
3,2,18179529,18179730,Afun_24,201,-
4,2,23084797,23084987,Afun_22,190,-


In [10]:
# additional data
comb_data = pd.concat(comb_data.values())
comb_data.shape
sa = comb_data['id'].str.split('_', expand=True)
comb_data['species'] = sa[0]
comb_data['amplicon'] = sa[1].astype(int)
comb_data['annotation'] = sa[1].replace(annot)
comb_data['color'] = comb_data['annotation'].replace(ANNOT_COLORS)
# chromosome names to be unique between species
comb_data['chrom'] = comb_data['species'] + '_' + comb_data['chrom']
# shift
comb_data.head()

,chrom,start,end,id,len,strand,species,amplicon,annotation,color
0,Aalb_2L,1067926,1068146,Aalb_53,220,-,Aalb,53,exon,orangered
1,Aalb_2L,5419554,5419760,Aalb_47,206,+,Aalb,47,intron,gold
2,Aalb_2L,8671327,8671500,Aalb_44,173,-,Aalb,44,intergenic,royalblue
3,Aalb_2L,10922763,10922958,Aalb_40,195,-,Aalb,40,intron,gold
4,Aalb_2L,21336966,21337166,Aalb_39,200,-,Aalb,39,intergenic,royalblue


## Prepare input files

In [11]:
# test chromosome
chrom = '2L'

In [12]:
def to_tsv(f, d):
    '''common writing - beds and blocks'''
    d.to_csv(f,
            sep='\t',
            header=False,
            index=False)

In [13]:
# get amplicons on chromosome
chrom_ampls = comb_data.loc[comb_data.chrom == 'Agam_' + chrom, 'amplicon']
chrom_ampls

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
Name: amplicon, dtype: int64

In [14]:
# subset data to chromosome
chrom_data = comb_data[comb_data.amplicon.isin(chrom_ampls)].copy()
chrom_data.shape

(27, 10)

In [15]:
# make blocks longer for visualisation, resolve overlaps
def longer_blocks(starts, ends, target_len, chrom_len=None):
    '''
    Make blocks longer shifting left if overlapping
    '''
    
    shift = int(target_len / 2)
    # gap introduced for when plotting overlapping regions
    ovl_gap = 0.1
        
    # initial enlargement
    corr_starts = starts - shift
    corr_ends = ends + shift
    
    # overlap removal
    nov_starts = list()
    nov_ends = list()
    ov = list()
    for s,e in zip(corr_starts, corr_ends):
        # init overlap analysis
        if len(ov) == 0:
            ov = [[s,e]]
            continue
        # no overlap: save previous region, store current
        if s >= ov[-1][-1]:
            for (novs, nove) in ov:
                nov_starts.append(novs)
                nov_ends.append(nove)
            ov = [[s,e]]
        # overlap - add current region, spread
        else:
            ov.append([s,e])
            # coordinates of the resulting stack of overlapping regions
            center = (ov[0][0] + ov[-1][-1]) / 2
            # adjust to chromosome ends
            center = max(center, shift * len(ov) + 1)
            if chrom_len:
                center = min(center, chrom_len - shift * len(ov))
            new_start = int(center) - shift * len(ov)
            # apply shift, leaving small gap between amplicons
            for i in range(len(ov)):
                # start
                ov[i][0] = int(new_start + shift * i * 2)
                # end 
                
                ov[i][1] = int(new_start + shift * (i + 1 - ovl_gap) * 2)
    # store remaining coordinates
    for (novs, nove) in ov:
        nov_starts.append(novs)
        nov_ends.append(nove)
    
    # trim starts and ends to fit chromosome
#     if nov_starts[0] < 1:
#         nov_starts[0] = 1
#     if chrom_len:
#         if nov_ends[-1] > chrom_len:
#             nov_ends[-1] = chrom_len
    
    return (nov_starts, nov_ends)
gam_cdata = chrom_data[chrom_data.species=='Agam'].copy()

corr_coords = longer_blocks(gam_cdata['start'],
                            gam_cdata['end'],
                            REGSIZE,
                            AGAM_CHROM_SIZES[chrom])
gam_cdata['corr_start'] = corr_coords[0]
gam_cdata['corr_end'] = corr_coords[1]
display(gam_cdata[['start','end','corr_start','corr_end']])

,start,end,corr_start,corr_end
0,10356417,10356608,9357032,10257032
1,10357458,10357648,10357032,11257032
2,12084495,12084690,11584495,12584690
3,13862864,13863055,13362864,14363055
4,18693481,18693714,17879422,18779422
5,19065171,19065363,18879422,19779422
6,20840388,20840620,20340388,21340620
7,26760700,26760890,26217768,27117768
8,27674646,27674836,27217768,28117768
9,40854755,40854948,40354755,41354948


In [16]:
# apply shift to all species
def longer_allsp(cdata, chrom_len):
    corr_data = list()
    for sp in SPP:
        clen = chrom_len if sp == 'Agam' else None
        sp_chrom_data = chrom_data[chrom_data.species==sp].sort_values('start')
        # do not parse empty coordinate lists
        if sp_chrom_data.shape[0] > 0:
            corr_coords = longer_blocks(sp_chrom_data['start'],
                                        sp_chrom_data['end'],
                                        REGSIZE,
                                        clen)
            sp_chrom_data['corr_start'] = corr_coords[0]
            sp_chrom_data['corr_end'] = corr_coords[1]
            corr_data.append(sp_chrom_data)
    return pd.concat(corr_data)
corr_data = longer_allsp(chrom_data, AGAM_CHROM_SIZES[chrom])
corr_data.head()

,chrom,start,end,id,len,strand,species,amplicon,annotation,color,corr_start,corr_end
0,Agam_2L,10356417,10356608,Agam_0,191,+,Agam,0,intron,gold,9357032,10257032
1,Agam_2L,10357458,10357648,Agam_1,190,+,Agam,1,exon,orangered,10357032,11257032
2,Agam_2L,12084495,12084690,Agam_2,195,+,Agam,2,intergenic,royalblue,11584495,12584690
3,Agam_2L,13862864,13863055,Agam_3,191,+,Agam,3,intergenic,royalblue,13362864,14363055
4,Agam_2L,18693481,18693714,Agam_4,233,+,Agam,4,exon,orangered,17879422,18779422


In [17]:
# add chromosome ends
def add_ce(cdata, chrom, chrom_data):
    color = 'gainsboro'
    ce = []
    for i, (chrom, start, end) in enumerate(chrom_data):
        species = chrom.split('_')[0]
        ce.extend([
            # start 
            {'chrom':chrom,
            'corr_start':start,
            'corr_end':start + 1,
            'id':species + '_start',
            'amplicon':1000 - 2 * (i + 1),
            'species':species,
            'color':color},
            # end
            {'chrom':chrom,
            'corr_start':end - 1,
            'corr_end':end,
            'id':species + '_end',
            'amplicon':1000 - 2 * (i + 1) + 1,
            'species':species,
            'color':color}
        ])
    return cdata.append(pd.DataFrame(ce), sort=False)
corr_data = add_ce(corr_data, chrom, CHROM_ENDS[chrom])
corr_data.tail(10)

,chrom,start,end,id,len,strand,species,amplicon,annotation,color,corr_start,corr_end
33,Afun_3,18114432.0,18114622.0,Afun_6,190.0,-,Afun,6,intergenic,royalblue,17614432,18614622
35,Afun_3,25816570.0,25816748.0,Afun_8,178.0,+,Afun,8,intron,gold,25316570,26316748
36,Afun_3,28805613.0,28805828.0,Afun_7,215.0,+,Afun,7,intron,gold,28305613,29305828
37,Afun_3,41162360.0,41162521.0,Afun_2,161.0,-,Afun,2,intergenic,royalblue,40662360,41662521
0,Agam_2L,NaN,NaN,Agam_start,NaN,NaN,Agam,998,NaN,gainsboro,1,2
1,Agam_2L,NaN,NaN,Agam_end,NaN,NaN,Agam,999,NaN,gainsboro,49364324,49364325
2,Afun_3,NaN,NaN,Afun_start,NaN,NaN,Afun,996,NaN,gainsboro,1,2
3,Afun_3,NaN,NaN,Afun_end,NaN,NaN,Afun,997,NaN,gainsboro,45633783,45633784
4,Aalb_3L,NaN,NaN,Aalb_start,NaN,NaN,Aalb,994,NaN,gainsboro,1,2
5,Aalb_3L,NaN,NaN,Aalb_end,NaN,NaN,Aalb,995,NaN,gainsboro,33601327,33601328


In [18]:
# BED files
to_tsv(COMB_BED.format(chrom),
       corr_data[['chrom','corr_start','corr_end','id','color']])

In [19]:
# multiway blocks
def write_blocks(cdata, chrom):
    block_data = cdata.pivot(index='amplicon', columns='species', values='id') \
                    .fillna('.')
    aligned_spp = [sp for sp in SPP if sp in block_data.columns]
    block_data = block_data[aligned_spp]
    # highlight thy synteny relations, otherwise the markers will be dim
    block_data['Agam'] = 'gainsboro*' + block_data['Agam']
    to_tsv(BLOCKS.format(chrom),
           block_data)
write_blocks(corr_data, chrom)
! head {BLOCKS.format(chrom)}

gainsboro*Agam_0	Aalb_0	.
gainsboro*Agam_1	.	.
gainsboro*Agam_2	.	Afun_2
gainsboro*Agam_3	Aalb_3	Afun_3
gainsboro*Agam_4	Aalb_4	Afun_4
gainsboro*Agam_5	.	Afun_5
gainsboro*Agam_6	.	Afun_6
gainsboro*Agam_7	Aalb_7	Afun_7
gainsboro*Agam_8	Aalb_8	Afun_8
gainsboro*Agam_9	Aalb_9	.


In [20]:
# display config
def write_disp(cdata, chrom):
    
    # scale all chromosomes relative to max
    ratio = float(AGAM_CHROM_SIZES[chrom]) / max(AGAM_CHROM_SIZES.values())
    # get chromosomes for other species, assume all markers on the same chromsome
    # funestus assembly does not have arms, use pre-defined arm mapping
    fun_chrom = GAM_TO_FUN_CHROM[chrom]
    # all species present in alignment 
    try:
        # albimanus chromosome from assembly header
        alb_chrom = cdata.loc[cdata.species == 'Aalb','chrom'].iloc[0].split('_')[-1]
        with open(DISP_CONF.format(chrom), 'w') as o:
            o.write("""# x,   y, rotation,     ha,     va, color, ratio,            label
0.5, 0.5,        0, center,    top,      k,     {ratio},       An. gambiae {}
0.5, 0.65,        0, center, top,      k,     {ratio},       An. albimanus {}
0.5, 0.35,        0, center, bottom,      k,     {ratio},        An. funestus {}
# edges
e, 0, 1
e, 0, 2
""".format(chrom, alb_chrom, fun_chrom, ratio=ratio))
    # homology loss happens for alb X
    except:
        with open(DISP_CONF.format(chrom), 'w') as o:
            o.write("""# x,   y, rotation,     ha,     va, color, ratio,            label
0.5, 0.5,        0, center,    top,      k,     {ratio},       An. gambiae {}
0.5, 0.35,        0, center, bottom,      k,     {ratio},        An. funestus {}
# edges
e, 0, 1
""".format(chrom, fun_chrom, ratio=ratio))
write_disp(corr_data, chrom)
! head {DISP_CONF.format(chrom)}

# x,   y, rotation,     ha,     va, color, ratio,            label
0.5, 0.5,        0, center,    top,      k,     0.802083691303,       An. gambiae 2L
0.5, 0.65,        0, center, top,      k,     0.802083691303,       An. albimanus 3L
0.5, 0.35,        0, center, bottom,      k,     0.802083691303,        An. funestus 3R
# edges
e, 0, 1
e, 0, 2


## Run visualisation

In [21]:
! python -m jcvi.graphics.synteny --notex \
    {BLOCKS.format(chrom)} \
    {COMB_BED.format(chrom)} \
    {DISP_CONF.format(chrom)}

13:06:37 [base] Load file `data/comb.2L.bed`
13:06:37 [base] Load file `data/blocks.2L.txt`
13:06:37 [base] Load file `data/disp.2L.txt`
Column 0: Agam_start - Agam_end (Agam_2L:2-49364325)
  Agam_2L .. 14 (14) features .. +
Column 1: Aalb_start - Aalb_end (Aalb_3L:2-33601328)
  Aalb_3L .. 9 (9) features .. -
Column 2: Afun_start - Afun_end (Afun_3:2-45633784)
  Afun_3 .. 10 (10) features .. -
/Users/am60/malaria/malariagen/binder/conda/envs/75_synteny_plot/lib/python2.7/site-packages/matplotlib/font_manager.py:1333: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
13:06:38 [base] Figure saved to `data/blocks.2L.pdf` (2400px x 2100px)


# Apply to all chromosomes

In [22]:
for chrom, chrom_len in AGAM_CHROM_SIZES.items():
    # get amplicons on chromosome
    chrom_ampls = comb_data.loc[comb_data.chrom == 'Agam_'+ chrom, 'amplicon']
    # subset data to chromosome
    chrom_data = comb_data[comb_data.amplicon.isin(chrom_ampls)].copy()
    # check that only one chromosome per species was retained
    uniq_chr = chrom_data.chrom.drop_duplicates()
    for sp in SP_CAPTIONS.keys():
        sp_chroms = uniq_chr.str.startswith(sp)
        assert sp_chroms.sum() <= 1, \
            'Multiple chromosomes observed for {} in {}: {}'.format(sp, chrom, uniq_chr[sp_chroms].to_list()) 
    # enlarge regions
    corr_data = longer_allsp(chrom_data, chrom_len)
    # add chromosome regions
    corr_data = add_ce(corr_data, chrom, CHROM_ENDS[chrom])
    # BED files
    to_tsv(COMB_BED.format(chrom),
           corr_data[['chrom','corr_start','corr_end','id','color']])
    # multiway blocks
    write_blocks(corr_data, chrom)
    # display layout
    write_disp(corr_data, chrom)
    # run visualisation
    ! python -m jcvi.graphics.synteny --notex \
        {BLOCKS.format(chrom)} \
        {COMB_BED.format(chrom)} \
        {DISP_CONF.format(chrom)}

13:06:39 [base] Load file `data/comb.2L.bed`
13:06:39 [base] Load file `data/blocks.2L.txt`
13:06:39 [base] Load file `data/disp.2L.txt`
Column 0: Agam_start - Agam_end (Agam_2L:2-49364325)
  Agam_2L .. 14 (14) features .. +
Column 1: Aalb_start - Aalb_end (Aalb_3L:2-33601328)
  Aalb_3L .. 9 (9) features .. -
Column 2: Afun_start - Afun_end (Afun_3:2-45633784)
  Afun_3 .. 10 (10) features .. -
/Users/am60/malaria/malariagen/binder/conda/envs/75_synteny_plot/lib/python2.7/site-packages/matplotlib/font_manager.py:1333: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
13:06:40 [base] Figure saved to `data/blocks.2L.pdf` (2400px x 2100px)
13:06:42 [base] Load file `data/comb.3R.bed`
13:06:42 [base] Load file `data/blocks.3R.txt`
13:06:42 [base] Load file `data/disp.3R.txt`
Column 0: Agam_start - Agam_end (Agam_3R:2-53200684)
  Agam_3R .. 19 (19) features .. +
Column 1: Aalb_start - Aalb_end (Aalb_2

In [23]:
raise Exception('Analysis ended')

Exception: Analysis ended

## Sandbox - test visualisation

In [ ]:
import jcvi.graphics.synteny as sntn
import matplotlib.pyplot as plt

In [ ]:
! python -m jcvi.graphics.synteny # --notex test/test.blocks.txt test/test.bed test/test.disp.txt

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,1, figsize=(10,10))

sntn.Synteny(fig, ax, 
        datafile=BLOCKS.format(chrom), 
        bedfile=COMB_BED.format(chrom), 
        layoutfile=DISP_CONF.format(chrom))
#             switch=switch, tree=tree, extra_features=opts.extra,
#             scalebar=opts.scalebar)
fig.show();
# Synteny plotting code parts use both fig and ax to draw various elements
# so making subplots work requires quite a lot of changes to the jcvi module code